In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
import sys
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
sys.path.append(parent_dir)

from google import genai
from scripts.embedding_helpers import similarity

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

In [2]:
from scripts.helpers import get_only_generated_info, load_courses

courses, ctoi = load_courses("../data/generated")
courses_generated_info = get_only_generated_info(courses)

In [3]:
from scripts.helpers import nice_dict_print
from scripts.embedding_helpers import embed_nice_print
import matplotlib.pyplot as plt
import numpy as np

# sample_courses_names = ["IV111", "MB152", "IV109", "VB035", "PV021", "VV024", "PV168", "PB130", "PB152", "PB152zk", "PV066", "IB031"]
# sample_courses_ids = [ctoi[name] for name in sample_courses_names]
# samples_courses = [courses_generated_info[i] for i in sample_courses_ids]

# for i in sample_courses_ids:
#     print(nice_dict_print(courses_generated_info[i]))

# sample_embeds = [embed_nice_print(client.models, course).embeddings[0].values for course in samples_courses]

# # Plot similarities

# similarity_matrix = np.zeros((len(samples_courses), len(samples_courses)))
# for i in range(len(samples_courses)):
#     for j in range(len(samples_courses)):
#         similarity_matrix[i, j] = similarity(sample_embeds[i], sample_embeds[j])

# plt.imshow(similarity_matrix, interpolation='nearest', cmap='coolwarm')
# plt.xticks(range(len(samples_courses)), sample_courses_names, rotation=45)
# plt.yticks(range(len(samples_courses)), sample_courses_names)
# plt.colorbar()
# plt.show()

In [4]:
all_embeds = []

In [5]:
# for i, course in enumerate(courses):
#     if i < 301:
#         continue
#     if i % 10 == 0:
#         print(i)
#     all_embeds.append(embed_nice_print(client.models, course).embeddings[0].values)
#     if i % 100 == 0:
#         np.save(f"../data/embeddings/embeds{i}.npy", all_embeds)

In [6]:
import numpy as np

all_embeds = np.load(f"../data/embeddings/embeds.npy", allow_pickle=True)
print(len(all_embeds))

379


In [7]:
from scripts.embedding_helpers import recommend_based_on_liked_disliked

course_names_pos = ["IV109", "PV254"]
course_names_neg = ["MV008"]

# find most similar courses
recommended_course_indices, simlarities = recommend_based_on_liked_disliked(course_names_pos, course_names_neg, all_embeds, ctoi, 30)
for i, sim in zip(recommended_course_indices, simlarities):
    print(sim, courses[i]["NAME"], courses[i]["CODE"])
    print(nice_dict_print(courses_generated_info[i]))
    print("-------------------------------------------")


0.2459270520951219  Recommender Systems   PV254 
 recommender systems collaborative filtering content based evaluation metrics machine learning data mining user-user item-item svd amazon netflix google news youtube educational recommender systems project based  quite-theoretical extremely-useful very-interesting very-Humanities specific very-difficult very-multidisciplinary extremely-project-based very-creative  
-------------------------------------------
0.21390712485628763  Modeling and Simulation   IV109 
 computational modeling simulation complex systems agent based modeling game theory network analysis system dynamics feedback loops cellular automata genetic algorithms neural networks modeling applications data analysis system thinking model evaluation  quite-theoretical extremely-useful very-interesting very-Humanities specific very-difficult very-multidisciplinary extremely-project-based very-creative  
-------------------------------------------
0.14096258769231976  Project in